# Single Machine

## Architecture

<img src="./helpers/replication-single-machine.png" alt="drawing" width="600"/>

## Prerequisites

- Single linux machine with installed PostgreSQL

## Install Tools

In [ ]:
# Patroni
apt install patroni -y

# ETCD
apt install curl -y
export ETCD_VER=v3.5.13
export DOWNLOAD_URL=https://github.com/etcd-io/etcd/releases/download
curl -L ${DOWNLOAD_URL}/${ETCD_VER}/etcd-${ETCD_VER}-linux-amd64.tar.gz -o /tmp/etcd-${ETCD_VER}-linux-amd64.tar.gz
tar xzvf /tmp/etcd-${ETCD_VER}-linux-amd64.tar.gz -C /usr/local/bin --strip-components=1

## Configure Patroni

### Single Node Config

Explore this `YAML` configuration file for Patroni.

This is a template for a basic Patroni deployment for a single Patroni - PostgreSQL pair.

*Configuration Explanation By Scope:*
- Global
    - Scope - Cluster name
    - Namespace - Where the data about the cluster is stored on the DCS (in our case ETCD)
    - Name - This deployment name
- restapi - Configurations for Patroni REST API
- ctl - Configurations for Patroni CTL API (script)
- etcd - Config connectivity to ETCD
- bootstrap - Configurations for the bootstrap of a PostgresSQL server, since Patroni is responsible to start a failed or non-existent server it needs PG parameters for that. \
That is complemented by the global `dcs` configurations presented in the next yaml block
- postgresql - This deployment explicit configurations
- watchdog - Can be used to make sure more than one Postgres node is not elected to be primary due to Patroni bug / error
- tags - Patroni tags to change some behaviors of the deployment, some examples attached

```yaml
scope: batman
namespace: /service
name: postgresql0

restapi:
  listen: 127.0.0.1:8008
  connect_address: 127.0.0.1:8008

ctl:
 insecure: false # Allow connections to Patroni REST API without verifying certificates

etcd:
  host: 127.0.0.1:2379

# The bootstrap configuration. Works only when the cluster is not yet initialized.
# If the cluster is already initialized, all changes in the `bootstrap` section are ignored!
bootstrap:
  initdb:  
  - encoding: UTF8
  - data-checksums

postgresql:
  listen: 127.0.0.1:5432
  connect_address: 127.0.0.1:5432
  data_dir: data/postgresql0
  authentication:
    replication:
      username: replicator
      password: rep-pass
    superuser:
      username: postgres
      password: zalando
    rewind:
      username: rewind_user
      password: rewind_password
# Can be used to make sure more than one Postgres node is not elected to be primary due to Patroni bug / error
# By default on
watchdog: 
  mode: off

tags:
    noloadbalance: false # Only HA
    nostream: false # Change to file based continuous recovery
    # replicatefrom: # Create cascading replication with this
```

### Global Config

This `YAML` block can be entered a single time in one config file across all the cluster since it's stored in ETCD and shared globally across all nodes as GLOBAL Variables.

Specific deployment variables can override this!

*Note* - Pay attention to the `parameters`, commented out are defaults.

```yaml
bootstrap:
  # This section will be written into Etcd:/<namespace>/<scope>/config after initializing new cluster
  # and all other cluster members will use it as a `global configuration`.
  # WARNING! If you want to change any of the parameters that were set up
  # via `bootstrap.dcs` section, please use `patronictl edit-config`!
  dcs:
    ttl: 30
    loop_wait: 10
    retry_timeout: 10
    maximum_lag_on_failover: 1048576
#    primary_start_timeout: 300
#    synchronous_mode: false
    postgresql:
      use_pg_rewind: true
      pg_hba:
      - host replication replicator 127.0.0.1/32 md5
      - host all all 0.0.0.0/0 md5
      parameters:
#        wal_level: hot_standby
#        hot_standby: "on"
#        max_connections: 100
#        max_worker_processes: 8
#        wal_keep_segments: 8
#        max_wal_senders: 10
#        max_replication_slots: 10
#        max_prepared_transactions: 0
#        max_locks_per_transaction: 64
#        wal_log_hints: "on"
#        track_commit_timestamp: "off"
#        archive_mode: "on"
#        archive_timeout: 1800s
#        archive_command: mkdir -p ../wal_archive && test ! -f ../wal_archive/%f && cp %p ../wal_archive/%f
#      recovery_conf:
#        restore_command: cp ../wal_archive/%f %p
```

### Add Configuration Files

1. Change to postgres user
1. cd to ~
1. Create 3 files:
    - postgres0.yml with global config
    - postgres1.yml -> change names and ports accordingly
    - postgres2.yml -> change names and ports accordingly

## Start ETCD

From a designated terminal start the ETCD service

In [ ]:
etcd --data-dir=data/etcd --enable-v2=true

## Start Patroni

Open 3 different terminals using `postgres` user in the directory `~` and start 3 deployment based on the config files we configured earlier

In [ ]:
patroni postgres0.yml
patroni postgres1.yml
patroni postgres2.yml

## Monitor Activity

In [ ]:
curl -s http://localhost:8008/patroni | jq .
curl -s http://localhost:8008/cluster | jq .
curl -s http://localhost:8008/history | jq .

## Play With Your HA Cluster

- Try killing some servers (check out `primary_start_timeout`)
- Try switching to file based log shipping
- Think of a way to use `pgBackRest` instead of `cp`
- Think how would periodic backup look like in HA